In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_non_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
train = train_features.drop('sig_id',axis=1)
test = test_features.drop('sig_id',axis=1)
target = train_targets_scored.drop('sig_id',axis=1)

In [ ]:
std_scaler = StandardScaler()

In [ ]:
train.cp_type = train.cp_type.map({'trt_cp':1,'ctl_vehicle':0})
train.cp_dose = train.cp_dose.map({'D1':0,'D2':1})

test.cp_type = test.cp_type.map({'trt_cp':1,'ctl_vehicle':0})
test.cp_dose = test.cp_dose.map({'D1':0,'D2':1})

In [ ]:
cor_matrix = train.corr().abs()

In [ ]:
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))

In [ ]:
thresh = 0.9
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > thresh)]
print(); print(to_drop)

In [ ]:
train_wo_mc = train.drop(to_drop,axis=1)
test_wo_mc = test.drop(to_drop,axis=1)

In [ ]:
std_scaler.fit(train_wo_mc)
train_wo_mc =std_scaler.transform(train_wo_mc)
test_wo_mc = std_scaler.transform(test_wo_mc)

In [ ]:
def get_log_loss(ytrue,pred):
    ytrue = ytrue.astype(np.float64)
    pred = pred.astype(np.float64) 
    score = 0
    for i in range(ytrue.shape[1]):
        _score = log_loss(ytrue[:,i], pred[:,i], labels=[0,1])
        score += _score / ytrue.shape[1]
        if str(_score) == 'nan':
            print(ytrue[:,i], pred[:,i])
            print(_score,score)
            print(i)
            break
    return score

In [ ]:
def get_model(ip_shape,op_shape):
    model  = tf.keras.Sequential([
        layers.Dense(ip_shape*2,input_shape=(ip_shape,)),
        layers.BatchNormalization(),
        layers.Activation('selu'),
        layers.Dropout(.4),
        layers.Dense(ip_shape*3),
        layers.BatchNormalization(),
        layers.Activation('selu'),
        layers.Dropout(.3),
        layers.Dense(op_shape,activation='softmax')
        ])
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics='accuracy')
    return model

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001,
  patience=3)

In [ ]:
# kfold = KFold(n_splits=3)
# Xdata = train_wo_mc.values
# ydata = target.values

In [ ]:
# def get_mean(data_list):
#     return sum(data_list)/len(data_list)

In [ ]:
# log_loss_score = []
# for train_idx, test_idx in kfold.split(Xdata):
#     xtrain,ytrain = Xdata[train_idx,:],ydata[train_idx,:]
#     xtest,ytest = Xdata[test_idx,:], ydata[test_idx,:]
    
#     model = get_model(xtrain.shape[1],ytrain.shape[1])
#     history =  model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50,batch_size=128,verbose=0,callbacks=[early_stop])
#     y_pred_test = model.predict(xtest)
#     log_loss_score.append(get_log_loss(ytest,y_pred_test))
    

In [ ]:
# sum(log_loss_score)/len(log_loss_score)

In [ ]:
train_wo_mc.shape, train.shape

In [ ]:
# Without K Fold
val_accuracy = []
log_loss_score =[]
y_pred_all = []
y_pred_all_test =[]
#for i in range(8,5,-1):
pca_n = [780,740,660]
for i in range(600,100,-20):
#for i in pca_n:
    print("I  ",i)
    pca = PCA(n_components=i)
    X = pca.fit_transform(train_wo_mc)
    Xt = pca.transform(test_wo_mc)
    xtrain, xtest,ytrain,ytest = train_test_split(X,target.values,test_size=.2)
    print(xtrain.shape[1],ytrain.shape[1])
    model = get_model(xtrain.shape[1],ytrain.shape[1])
    history = model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50,batch_size=128,verbose=0,callbacks=[early_stop])
    val_accuracy.append(history.history['val_accuracy'])
    y_pred = model.predict(xtest)
    log_loss_current = get_log_loss(ytest,y_pred)
    if log_loss_current < .0195:
        y_pred_test = model.predict(Xt)
        log_loss_score.append(log_loss_current)
        y_pred_all.append(y_pred)
        y_pred_all_test.append(y_pred_test)
        print("Mean Log_Loss",sum(log_loss_score)/len(log_loss_score))
    

In [ ]:
plt.plot(log_loss_score)


In [ ]:
(len(y_pred_all)+5)

In [ ]:
y_pred_sum = np.sum(y_pred_all,axis=0)/(len(y_pred_all)+5)
get_log_loss(ytest,y_pred_sum)

In [ ]:
y_pred_test_sum = np.sum(y_pred_all_test,axis=0)/float(len(y_pred_all))

In [ ]:
cols = target.columns.tolist()
#cols.remove('sig_id')

submission.loc[:,cols] = y_pred_test_sum

submission.to_csv("submission.csv",index=False)

In [ ]:
print("hi")